In [1]:
# Import Libraries
import pandas as pd

In [4]:
# Load alpha diversity metrics from QIIME2
# directories are going to be complicated -- I downloaded, unzipped, renamed, and moved everything to the main directory

shannon = pd.read_csv('shannon-alpha-diversity.tsv', sep='\t')
shannon.columns = ['sample-id','shannon_entropy']

faith = pd.read_csv('faith-alpha-diversity.tsv', sep='\t')
faith.columns = ['sample-id','faith_diversity']

pielou = pd.read_csv('pielou-alpha-diversity.tsv', sep='\t')
pielou.columns = ['sample-id','pielou_evenness']

observed = pd.read_csv('observed-features-alpha-diversity.tsv', sep='\t')
observed.columns = ['sample-id','observed_features']

# Load metadata from QIIME2 and append diversity metrics

metadata = pd.read_csv('metadata.tsv', sep='\t')
metadata = metadata.drop(metadata.index[0])
metadata = metadata.reset_index(drop=True)
metadata = metadata.merge(shannon,how='outer')
metadata = metadata.merge(faith,how='outer')
metadata = metadata.merge(pielou,how='outer')
metadata = metadata.merge(observed,how='outer')
metadata.head()

,sample-id,barcode-sequence,body-site,year,month,day,subject,reported-antibiotic-usage,days-since-experiment-start,shannon_entropy,faith_diversity,pielou_evenness,observed_features
0,L1S8,AGCTGACTAGTC,gut,2008,10,28,subject-1,Yes,0,3.167834,4.581991,0.608093,37.0
1,L1S57,ACACACTATGGC,gut,2009,1,20,subject-1,No,84,4.189647,5.799407,0.703646,62.0
2,L1S76,ACTACGTGTGGT,gut,2009,2,17,subject-1,No,112,3.534307,5.838537,0.617031,53.0
3,L1S105,AGTGCGATGCGT,gut,2009,3,17,subject-1,No,140,3.799282,6.385523,0.657161,55.0
4,L2S155,ACGATGCGACCA,left palm,2009,1,20,subject-1,No,84,5.033764,16.472200,0.764433,96.0


In [3]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px

df = metadata

app = dash.Dash(__name__)

app.layout = html.Div([
    html.P("x-axis:"),
    dcc.Checklist(
        id='x-axis', 
        options=[{'value': x, 'label': x}  
                 for x in df.columns],
        value=['subject'], 
        labelStyle={'display': 'inline-block'}
    ),
    html.P("y-axis:"),
    dcc.RadioItems(
        id='y-axis', 
        options=[{'value': x, 'label': x} 
                 for x in ['shannon_entropy','faith_diversity','pielou_evenness','observed_features']],
        value='shannon_entropy', 
        labelStyle={'display': 'inline-block'}
    ),
    dcc.Graph(id="box-plot"),
])

@app.callback(
    Output("box-plot", "figure"), 
    [Input("x-axis", "value"), 
     Input("y-axis", "value")])
def generate_chart(x, y):
    fig = px.box(df, x=x, y=y)
    return fig

app.run_server(debug=True, use_reloader=False)

Dash is running on http://x86_64-conda-linux-gnu:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


gaierror: [Errno -2] Name or service not known